<a href="https://colab.research.google.com/github/patriciagvas/Bootcamp_Soulcode_Analista_Dados/blob/main/Atividade_05_bucket_GCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Na GCP crie uma bucket (Cloud Storage) com qualquer nome e dentro dela crie uma
pasta com o nome de dados_bruto e dentro dele salve o arquivo (csv) com os dados do
da base escolhida no site https://dados.gov.br/, deixe o acesso público ativado

No colab faça a leitura desse arquivo e a partir dele crie diferentes insight (pandas/pyspark/numpy) com os resultados de agrupamentos e salve esses arquivos como csv e envie diretamente pelo cada um deles para o bucket dentro de uma pasta chamada "Dados_tratados"

In [ ]:
!pip install --upgrade google-cloud-storage google-cloud-bigquery

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.8.0
    Uninstalling google-cloud-storage-2.8.0:
      Successfully uninstalled google-cloud-storage-2.8.0


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import os
from google.cloud import storage
# Inicialize o cliente
client = storage.Client()
# Defina o nome do bucket
bucket_name = 'analista_de_dados_soulcode'
bucket = client.bucket(bucket_name)

# Caminho dentro do bucket (pasta)
destination_blob_path = 'dados_brutos/f-etanol-producao.csv'
# Faça o upload de um arquivo CSV
blob = bucket.blob(destination_blob_path)
blob.upload_from_filename('/content/drive/MyDrive/Colab Notebooks/Datasets/f-etanol-producao.csv') #caminho do arquivo csv que quero fazer upload


**Análises**: Com base nos dados obtidos pelo site gov avaliar a produção de etanol anidro e hidratado no Brasil de 2019 a 2023.

In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

arquivo = '/content/drive/MyDrive/Colab Notebooks/Datasets/f-etanol-producao.csv'
df_producao = pd.read_csv(arquivo)

df_producao = df_producao.drop_duplicates()
df_producao.isnull().sum()
df_producao.columns = df_producao.columns.str.title()
df_producao = df_producao.dropna()
#df_producao.head()

# Arquivo com dados de matéria-prima para produção de etanol
arquivo_mp = '/content/drive/MyDrive/Colab Notebooks/Datasets/h-etanol-materia-prima.csv'
df_mp = pd.read_csv(arquivo_mp)

df_mp = df_mp.drop_duplicates()
df_mp.isnull().sum()
df_mp.columns = df_mp.columns.str.title()
df_mp = df_mp.dropna()
df_mp.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Mês/Ano,Região,Estado,Produto,Quantidade Processada (T)
0,08/2023,SUDESTE,São Paulo,BAGAÇO OU PALHA DE CANA,12713
1,08/2023,CENTRO OESTE,Goiás,CANA DE AÇÚCAR,7422467
2,08/2023,CENTRO OESTE,Mato Grosso,CANA DE AÇÚCAR,1544723
3,08/2023,CENTRO OESTE,Mato Grosso do Sul,CANA DE AÇÚCAR,4378373
4,08/2023,NORDESTE,Alagoas,CANA DE AÇÚCAR,4857


**1** - Produção de etanol hidratado e anidro no Brasil por mês de 2019 a 2023

In [ ]:
producao_mensal_anidro = df_producao.groupby('Mês/Ano')['Produção Etanol Anidro (M³/D)'].sum().reset_index(name = "Produção Etanol Anidro (M³/D)")
resultado = round(producao_mensal_anidro, 2)
print(resultado)
producao_mensal_anidro.to_csv('producao_mensal_anidro.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_mensal_anidro.csv')
processed_blob.upload_from_filename('producao_mensal_anidro.csv')

      Mês/Ano  Produção Etanol Anidro (M³/D)
0  2019-01-01                       134879.0
1  2019-02-01                        67328.0
2  2019-03-01                       104452.0
3  2019-04-01                       422938.0
4  2019-05-01                      1247647.0
5  2019-06-01                      1504380.0
6  2019-07-01                      1552913.0
7  2019-08-01                      1568563.0
8  2019-09-01                      1337736.0
9  2019-10-01                      1412341.0
10 2019-11-01                       805905.0
11 2019-12-01                       235172.0
12 2020-01-01                       151268.0
13 2020-02-01                        75987.0
14 2020-03-01                       142132.0
15 2020-04-01                       615541.0
16 2020-05-01                      1060050.0
17 2020-06-01                      1184708.0
18 2020-07-01                      1469321.0
19 2020-08-01                      1450988.0
20 2020-09-01                      1617403.0
21 2020-10

In [ ]:
producao_mensal_hidratado = df_producao.groupby('Mês/Ano')['Produção Etanol Hidratado(M³/D)'].sum().reset_index(name = "Produção Etanol Hidratado (M³/D)")
resultado = round(producao_mensal_hidratado, 2)
print(resultado)
producao_mensal_hidratado.to_csv('producao_mensal_hidratado.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_mensal_hidratado.csv')
processed_blob.upload_from_filename('producao_mensal_hidratado.csv')

      Mês/Ano  Produção Etanol Hidratado (M³/D)
0  2019-01-01                          218427.0
1  2019-02-01                          158681.0
2  2019-03-01                          471761.0
3  2019-04-01                         1849021.0
4  2019-05-01                         2796146.0
5  2019-06-01                         3064382.0
6  2019-07-01                         3404262.0
7  2019-08-01                         3784037.0
8  2019-09-01                         3548800.0
9  2019-10-01                         3341073.0
10 2019-11-01                         1660643.0
11 2019-12-01                          492701.0
12 2020-01-01                          323719.0
13 2020-02-01                          236518.0
14 2020-03-01                          522999.0
15 2020-04-01                         1964575.0
16 2020-05-01                         2651434.0
17 2020-06-01                         2711582.0
18 2020-07-01                         3170644.0
19 2020-08-01                         31

**2** - Produção total e média de etanol hidratado e anidro por região do Brasil

In [ ]:
producao_total_hidratado = df_producao.groupby('Região')['Produção Etanol Hidratado(M³/D)'].sum().reset_index(name = "Produção Etanol Hidratado (M³/D)")
producao_total_hidratado = round(producao_total_hidratado, 2)
producao_total_hidratado = producao_total_hidratado.sort_values(by='Produção Etanol Hidratado (M³/D)', ascending = False)
print(producao_total_hidratado)
producao_total_hidratado.to_csv('producao_total_hidratado.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_total_hidratado.csv')
processed_blob.upload_from_filename('producao_total_hidratado.csv')

         Região  Produção Etanol Hidratado (M³/D)
3       SUDESTE                        47648732.0
0  CENTRO OESTE                        40778931.0
1      NORDESTE                         4432298.0
4           SUL                         3248073.0
2         NORTE                          475147.0


In [ ]:
producao_media_hidratado = df_producao.groupby('Região')['Produção Etanol Hidratado(M³/D)'].mean().reset_index(name = "Produção Média De Etanol Hidratado (M³/D)")
producao_media_hidratado = round(producao_media_hidratado, 2)
producao_media_hidratado = producao_media_hidratado.sort_values(by='Produção Média De Etanol Hidratado (M³/D)', ascending = False)
print(producao_media_hidratado)
producao_media_hidratado.to_csv('producao_media_hidratado.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_media_hidratado.csv')
processed_blob.upload_from_filename('producao_media_hidratado.csv')

         Região  Produção Média De Etanol Hidratado (M³/D)
3       SUDESTE                                  333207.92
0  CENTRO OESTE                                  248652.02
4           SUL                                   63687.71
1      NORDESTE                                   16001.08
2         NORTE                                    7091.75


In [ ]:
# Etanol Anidro

producao_total_anidro = df_producao.groupby('Região')['Produção Etanol Anidro (M³/D)'].sum().reset_index(name = "Produção Etanol Anidro (M³/D)")
producao_total_anidro = round(producao_total_anidro, 2)
producao_total_anidro = producao_total_anidro.sort_values(by='Produção Etanol Anidro (M³/D)', ascending = False)
print(producao_total_anidro)
producao_total_anidro.to_csv('producao_total_anidro.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_total_anidro.csv')
processed_blob.upload_from_filename('producao_total_anidro.csv')

         Região  Produção Etanol Anidro (M³/D)
3       SUDESTE                     31094888.0
0  CENTRO OESTE                     14847046.0
1      NORDESTE                      3975862.0
4           SUL                      2759378.0
2         NORTE                       611194.0


In [ ]:
producao_media_anidro = df_producao.groupby('Região')['Produção Etanol Anidro (M³/D)'].mean().reset_index(name = "Produção Média De Etanol Anidro(M³/D)")
producao_media_anidro = round(producao_media_anidro, 2)
producao_media_anidro = producao_media_anidro.sort_values(by='Produção Média De Etanol Anidro(M³/D)', ascending = False)
print(producao_media_anidro)
producao_media_anidro.to_csv('producao_media_anidro.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_media_anidro.csv')
processed_blob.upload_from_filename('producao_media_anidro.csv')

         Região  Produção Média De Etanol Anidro(M³/D)
3       SUDESTE                              217446.77
0  CENTRO OESTE                               90530.77
4           SUL                               54105.45
1      NORDESTE                               14353.29
2         NORTE                                9122.30


**3** - Estado com maior produção de etanol anidro e hidratado

In [ ]:
#Identificando a produção de cada estado
producao_estado_anidro= df_producao.groupby('Estado')['Produção Etanol Anidro (M³/D)'].sum().reset_index(name = "Produção De Etanol Anidro(M³/D)")
producao_estado_anidro = round(producao_estado_anidro, 2)
producao_estado_anidro = producao_estado_anidro.sort_values(by='Produção De Etanol Anidro(M³/D)', ascending = False)
print(producao_estado_anidro)
producao_estado_anidro.to_csv('producao_estado_anidro.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_estado_anidro.csv')
processed_blob.upload_from_filename('producao_estado_anidro.csv')

#Estado com maior e menor produção de etanol anidro

estado_maior_producao = producao_estado_anidro.index[0]
maior_producao = producao_estado_anidro.iloc[0]
print(f"O estado com a maior produção de etanol anidro é {estado_maior_producao} com uma produção total de {maior_producao:} m³.")

producao_estado_anidro_cresc = producao_estado_anidro.sort_values(by='Produção De Etanol Anidro(M³/D)', ascending = True)
estado_menor_producao = producao_estado_anidro_cresc.index[0]
menor_producao = producao_estado_anidro_cresc.iloc[0]
print(f"O estado com a menor produção de etanol anidro é {estado_menor_producao} com uma produção total de {menor_producao:} m³.")


                 Estado  Produção De Etanol Anidro(M³/D)
15            São Paulo                       25435649.0
5           Mato Grosso                        5752132.0
7          Minas Gerais                        5256391.0
3                 Goiás                        5046165.0
6    Mato Grosso do Sul                        4048749.0
8                Paraná                        2759378.0
9               Paraíba                         968830.0
0               Alagoas                         894576.0
4              Maranhão                         665086.0
11           Pernambuco                         567730.0
1                 Bahia                         546334.0
16            Tocantins                         426072.0
2        Espírito Santo                         402848.0
10                 Pará                         185122.0
13  Rio Grande do Norte                         135056.0
12                Piauí                         111820.0
14              Sergipe        

In [ ]:
#Identificando a produção de cada estado
producao_estado_hidratado= df_producao.groupby('Estado')['Produção Etanol Hidratado(M³/D)'].sum().reset_index(name = "Produção De Etanol Hidratado(M³/D)")
producao_estado_hidratado = round(producao_estado_hidratado, 2)
producao_estado_hidratado = producao_estado_hidratado.sort_values(by='Produção De Etanol Hidratado(M³/D)', ascending = False)
print(producao_estado_hidratado)
producao_estado_hidratado.to_csv('producao_estado_hidratado.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_estado_hidratado.csv')
processed_blob.upload_from_filename('producao_estado_hidratado.csv')

#Estado com maior e menor produção de etanol hidratado

estado_maior_producao = producao_estado_hidratado.index[0]
maior_producao = producao_estado_hidratado.iloc[0]
print(f"O estado com a maior produção de etanol hidratado é {estado_maior_producao} com uma produção total de {maior_producao:} m³.")

producao_estado_hidratado_cresc = producao_estado_hidratado.sort_values(by='Produção De Etanol Hidratado(M³/D)', ascending = True)
estado_menor_producao = producao_estado_hidratado_cresc.index[0]
menor_producao = producao_estado_hidratado_cresc.iloc[0]
print(f"O estado com a menor produção de etanol hidratado é {estado_menor_producao} com uma produção total de {menor_producao:} m³.")

                 Estado  Produção De Etanol Hidratado(M³/D)
15            São Paulo                          38381393.0
3                 Goiás                          19149443.0
5           Mato Grosso                          11314370.0
6    Mato Grosso do Sul                          10315118.0
7          Minas Gerais                           9206469.0
8                Paraná                           3248073.0
0               Alagoas                           1096037.0
11           Pernambuco                            973878.0
1                 Bahia                            848054.0
9               Paraíba                            685686.0
16            Tocantins                            412777.0
14              Sergipe                            342959.0
13  Rio Grande do Norte                            333330.0
4              Maranhão                             76919.0
12                Piauí                             75435.0
10                 Pará                 

**4** - Época com maior produção de cada tipo de etanol

In [ ]:
producao_epoca_hidratado= df_producao.groupby('Mês/Ano')['Produção Etanol Hidratado(M³/D)'].max().reset_index(name = "Produção De Etanol Hidratado(M³/D)")
producao_epoca_hidratado = round(producao_epoca_hidratado, 2)
producao_epoca_hidratado = producao_epoca_hidratado.sort_values(by='Produção De Etanol Hidratado(M³/D)', ascending = False)
print(producao_epoca_hidratado)
producao_epoca_hidratado.to_csv('producao_epoca_hidratado.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_epoca_hidratado.csv')
processed_blob.upload_from_filename('producao_epoca_hidratado.csv')

      Mês/Ano  Produção De Etanol Hidratado(M³/D)
7  2019-08-01                           1758127.0
9  2019-10-01                           1582724.0
6  2019-07-01                           1571070.0
8  2019-09-01                           1556298.0
18 2020-07-01                           1475375.0
5  2019-06-01                           1459884.0
19 2020-08-01                           1459643.0
16 2020-05-01                           1390115.0
4  2019-05-01                           1354345.0
20 2020-09-01                           1350952.0
17 2020-06-01                           1292693.0
31 2021-08-01                           1135592.0
28 2021-05-01                           1134782.0
42 2022-07-01                           1132409.0
30 2021-07-01                           1110452.0
55 2023-08-01                           1105928.0
54 2023-07-01                           1075795.0
32 2021-09-01                           1067366.0
21 2020-10-01                           1026410.0


In [ ]:
producao_epoca_anidro= df_producao.groupby('Mês/Ano')['Produção Etanol Anidro (M³/D)'].max().reset_index(name = "Produção De Etanol Anidro(M³/D)")
producao_epoca_anidro = round(producao_epoca_anidro, 2)
producao_epoca_anidro = producao_epoca_anidro.sort_values(by='Produção De Etanol Anidro(M³/D)', ascending = False)
print(producao_epoca_anidro)
producao_epoca_anidro.to_csv('producao_epoca_anidro.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_epoca_anidro.csv')
processed_blob.upload_from_filename('producao_epoca_anidro.csv')

      Mês/Ano  Produção De Etanol Anidro(M³/D)
30 2021-07-01                        1008391.0
42 2022-07-01                         970906.0
31 2021-08-01                         960479.0
54 2023-07-01                         953919.0
52 2023-05-01                         912923.0
55 2023-08-01                         912080.0
7  2019-08-01                         904786.0
6  2019-07-01                         888342.0
32 2021-09-01                         887598.0
43 2022-08-01                         880173.0
5  2019-06-01                         870679.0
29 2021-06-01                         863876.0
20 2020-09-01                         815876.0
41 2022-06-01                         810447.0
28 2021-05-01                         789325.0
53 2023-06-01                         778336.0
4  2019-05-01                         770211.0
9  2019-10-01                         769479.0
18 2020-07-01                         757806.0
8  2019-09-01                         751201.0
44 2022-09-01

**5** - Produção anual de cada tipo de etanol de 2019 a 2023

In [ ]:
df_producao['Mês/Ano'] = pd.to_datetime(df_producao['Mês/Ano'], format='%d%m/%Y')  # ajuste o formato conforme necessário
df_producao['Ano'] = df_producao['Mês/Ano'].dt.year
#print(df_producao)
df_producao.to_csv('df_producao_ano.csv', index=False)

producao_anual_anidro= df_producao.groupby('Ano')['Produção Etanol Anidro (M³/D)'].sum().reset_index(name = "Produção De Etanol Anidro(M³/D)")
producao_anual_anidro = round(producao_anual_anidro, 2)
producao_anual_anidro = producao_anual_anidro.sort_values(by='Produção De Etanol Anidro(M³/D)', ascending = False)
print(producao_anual_anidro)
producao_anual_anidro.to_csv('producao_anual_anidro.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_anual_anidro.csv')
processed_blob.upload_from_filename('producao_anual_anidro.csv')


    Ano  Produção De Etanol Anidro(M³/D)
3  2022                       12322885.0
2  2021                       11417120.0
0  2019                       10394254.0
1  2020                       10235081.0
4  2023                        8919028.0


In [ ]:
df_producao['Mês/Ano'] = pd.to_datetime(df_producao['Mês/Ano'], format='%d%m/%Y')  # ajuste o formato conforme necessário
df_producao['Ano'] = df_producao['Mês/Ano'].dt.year
#print(df_producao)
df_producao.to_csv('df_producao_ano.csv', index=False)

producao_anual_hidratado= df_producao.groupby('Ano')['Produção Etanol Hidratado(M³/D)'].sum().reset_index(name = "Produção De Etanol Hidratado(M³/D)")
producao_anual_hidratado = round(producao_anual_hidratado, 2)
producao_anual_hidratado = producao_anual_hidratado.sort_values(by='Produção De Etanol Hidratado(M³/D)', ascending = False)
print(producao_anual_hidratado)
producao_anual_anidro.to_csv('producao_anual_hidratado.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_anual_hidratado.csv')
processed_blob.upload_from_filename('producao_anual_hidratado.csv')

    Ano  Produção De Etanol Hidratado(M³/D)
0  2019                          24789934.0
1  2020                          22122860.0
2  2021                          18498825.0
3  2022                          18228960.0
4  2023                          12942602.0
